# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
#import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
#lets me define my own scoring method
from sklearn.metrics import make_scorer 

#lets me add random_state to cross_val_score so i get the same scores each time
from sklearn.model_selection import StratifiedKFold

extracting x label from the dataframe

In [11]:
x_train=df_train['story']
y_train = df_train['gender']

funtion that remove white space from the stories

In [12]:
def clean_text(text):
    # Remove numbers , spaces and leading and trailing spaces
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

function that create tokens out of the stories

In [13]:
def tokenize(text):
    tokens = text.split()
    return tokens

In [14]:
cleaned_x_train =x_train.apply(clean_text)
tokenized_x_train = cleaned_x_train.apply(tokenize)

function that check which base model is the best with combination of which vectorizer is the best

In [33]:
def find_best_model(data, labels):
    
    models = [DecisionTreeClassifier(), LinearSVC(),MLPClassifier(),Perceptron(),
              SGDClassifier(),KMeans(),MultinomialNB(),GaussianNB(),KNeighborsClassifier()]
    
    vectorizers = [CountVectorizer(tokenizer=lambda doc: doc, lowercase=False), TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)]
    cv = StratifiedKFold(shuffle=True,random_state = 5)

    best_score = 0
    best_model = None
    best_vectorizer = None

    for vectorizer in vectorizers:
        X = vectorizer.fit_transform(data)
        X = preprocessing.normalize(X, norm='max')
        for model in models:
            score = cross_val_score(model, X, labels, cv=cv,n_jobs=-1, scoring=make_scorer(f1_score, average='macro')).mean()
            if score > best_score:
                best_score = score
                best_model = model
                best_vectorizer = vectorizer

    return best_model, best_vectorizer, best_score

In [35]:
model,vectorizer,score  = find_best_model(tokenized_x_train, y_train)
model
vectorizer  
score

SGDClassifier()

CountVectorizer(lowercase=False,
                tokenizer=<function find_best_model.<locals>.<lambda> at 0x00000134A3BA7700>)

0.684652850015974

using girdsearchCV to check the best parameters for the model I found earlier to be the best in order to imporve on it

In [19]:
vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
vectorized_x_train = vectorizer.fit_transform(tokenized_x_train)
normalized_x_train = preprocessing.normalize(vectorized_x_train, norm='max')

#created a vocabulary that includes the features from the training, since new data won't have the same
#amount features and thus won't be testable
vocab = vectorizer.get_feature_names()
vectorizer_new = CountVectorizer(vocabulary=vocab, tokenizer=lambda doc: doc, lowercase=False)

In [21]:
params = {
    'loss': ['hinge', 'log', 'modified_huber'],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'alpha': [0.00001,0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0,0.1, 0.01, 0.001]
}
cv = StratifiedKFold(shuffle=True,random_state = 5)
clf = GridSearchCV(SGDClassifier(), params,cv=cv, n_jobs=-1, scoring=make_scorer(f1_score, average='macro'))
clf.fit(normalized_x_train,y_train)
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=5, shuffle=True),
             estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01],
                         'eta0': [0, 0.1, 0.01, 0.001],
                         'learning_rate': ['constant', 'optimal', 'invscaling',
                                           'adaptive'],
                         'loss': ['hinge', 'log', 'modified_huber'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none']},
             scoring=make_scorer(f1_score, average=macro))

best parameter set is: {'alpha': 0.0001, 'eta0': 0.001, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'penalty': 'l1'}  and its score was 0.6982221212869263


creating a new model based on all of what I found earlier and training it, in order to use it on my test data

In [22]:
final_model = SGDClassifier(alpha = 0.0001, eta0 = 0.001, learning_rate = 'optimal',loss = 'modified_huber', penalty = 'l1', random_state=5)
final_model.fit(normalized_x_train, y_train)

x_test=df_test['story']
cleaned_x_test =x_test.apply(clean_text)
tokenized_x_test = cleaned_x_test.apply(tokenize)
vectorized_x_test = vectorizer_new.fit_transform(tokenized_x_test)
normalized_x_test = preprocessing.normalize(vectorized_x_test, norm='max')

predict = final_model.predict(normalized_x_test)
predict

SGDClassifier(eta0=0.1, learning_rate='constant', random_state=5, tol=1e-05)

array(['m', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'f', 'm', 'f',
       'm', 'f', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'f', 'f', 'm', 'f', 'm',
       'm', 'f', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
       'f', 'f', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'f', 'f',
       'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'f', 'f', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm',
       'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'f', 'm', 'm',
       'm', 'm', 'm', 'm', 'f', 'f', 'm', 'm', 'm', 'm', 'm', 'f', 'f',
       'f', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'f', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm

converting the predictions into pandas dataframe

In [23]:
df=pd.DataFrame({"test_example_id":df_test['test_example_id'],"Predicted":predict})


first 5 predictions on test data

In [24]:
df.head(5)

,test_example_id,Predicted
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


last 5 predictions on test data

In [25]:
df.tail(5)

,test_example_id,Predicted
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [36]:
#df_predicted.to_csv('classification_results.csv',index=False)